**Dual LSTM Encoder for Dialog Response Generation**

http://www.wildml.com/2016/07/deep-learning-for-chatbots-2-retrieval-based-model-tensorflow/

https://github.com/dennybritz/chatbot-retrieval

https://github.com/rkadlec/ubuntu-ranking-dataset-creator

https://arxiv.org/abs/1506.08909

In [ ]:
import tensorflow as tf
tf.VERSION

**Experiment / Estimator**

https://www.tensorflow.org/versions/r1.2/api_docs/python/tf/contrib/learn/Experiment

https://www.tensorflow.org/versions/r1.2/api_docs/python/tf/estimator/Estimator

https://www.tensorflow.org/versions/r1.2/api_docs/python/tf/estimator

**model_fn**
```text
 |        model_fn: Model function. Follows the signature:
 |      
 |          * Args:
 |      
 |            * `features`: This is the first item returned from the `input_fn`
 |                   passed to `train`, 'evaluate`, and `predict`. This should be a
 |                   single `Tensor` or `dict` of same.
 |            * `labels`: This is the second item returned from the `input_fn`
 |                   passed to `train`, 'evaluate`, and `predict`. This should be a
 |                   single `Tensor` or `dict` of same (for multi-head models). If
 |                   mode is `ModeKeys.PREDICT`, `labels=None` will be passed. If
 |                   the `model_fn`'s signature does not accept `mode`, the
 |                   `model_fn` must still be able to handle `labels=None`.
 |            * `mode`: Optional. Specifies if this training, evaluation or
 |                   prediction. See `ModeKeys`.
 |            * `params`: Optional `dict` of hyperparameters.  Will receive what
 |                   is passed to Estimator in `params` parameter. This allows
 |                   to configure Estimators from hyper parameter tuning.
 |            * `config`: Optional configuration object. Will receive what is passed
 |                   to Estimator in `config` parameter, or the default `config`.
 |                   Allows updating things in your model_fn based on configuration
 |                   such as `num_ps_replicas`, or `model_dir`.
 |      
 |          * Returns:
 |            `EstimatorSpec`
 ```

In [ ]:
def dual_encoder(vocab_size,
                 embed_size,
                 hidden_size,
                 input_context,
                 input_context_len,
                 input_utterance,
                 input_utterance_len,
                 targets):

    with tf.variable_scope('embedding'):
        embeddings = tf.get_variable(
            'embeddings',
            shape=(vocab_size, embed_size),
            initializer=tf.random_uniform_initializer(-0.25, 0.25))

        context_embed = tf.nn.embedding_lookup(
            embeddings, input_context, name='context_embed')
        utterance_embed = tf.nn.embedding_lookup(
            embeddings, input_utterance, name='utterance_embed')

        input_embed = tf.concat([context_embed, utterance_embed], axis=0)
        input_length = tf.concat([input_context_len, input_utterance_len], axis=0)
        input_length = tf.reshape(input_length, [-1])

    with tf.variable_scope('rnn'):
        cell = tf.nn.rnn_cell.LSTMCell(
            hidden_size,
            forget_bias=2.0,
            use_peepholes=True,
            state_is_tuple=True)

        outputs, states = tf.nn.dynamic_rnn(
            cell,
            input_embed,
            sequence_length=input_length,
            dtype=tf.float32)

        context_encoding, utterance_encoding = tf.split(
            states.h, num_or_size_splits=2, axis=0)

    with tf.variable_scope('prediction'):
        ct = context_encoding
        rt = utterance_encoding
        M = tf.get_variable(
            'M',
            shape=(hidden_size, hidden_size),
            initializer=tf.truncated_normal_initializer())

        ct_M = tf.matmul(ct, M)
        batch_ct_M = tf.expand_dims(ct_M, axis=2)
        batch_rt = tf.expand_dims(rt, axis=2)
        batch_ct_M_r = tf.matmul(batch_ct_M, batch_rt, transpose_a=True)
        ct_M_r = tf.squeeze(batch_ct_M_r, axis=2)

        b = tf.get_variable(
            'b', shape=(), initializer=tf.zeros_initializer())
        
        logits = ct_M_r + b
        
        probs = tf.sigmoid(logits)

    if targets is None:
        return probs, None

    loss = tf.losses.sigmoid_cross_entropy(multi_class_labels=tf.to_float(targets), logits=logits)
    loss = tf.reduce_mean(loss, name="loss")
    
    return probs, loss

In [ ]:
def model_fn(features, labels, mode, params):
    vocab_size = params['vocab_size']
    embed_size = params['embed_size']
    hidden_size = params['hidden_size']

    input_context = features['context']
    input_context_len = features['context_len']
    input_utterance = features['utterance']
    input_utterance_len = features['utterance_len']

    probs, loss = dual_encoder(
        vocab_size,
        embed_size,
        hidden_size,
        input_context,
        input_context_len,
        input_utterance,
        input_utterance_len,
        labels)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        _learning_rate = params['learning_rate']
        _optimizer =  params['optimizer']
        
        train_op = tf.contrib.layers.optimize_loss(
            loss=loss,
            global_step=tf.contrib.framework.get_global_step(),
            learning_rate=_learning_rate,
            clip_gradients=10.0,
            optimizer=_optimizer)
    else:
        train_op = None
    
    return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=probs,
        loss=loss,
        train_op=train_op)

**Input**

In [ ]:
# `tokenizer` function must be defined before restoring the vocabulary object
# (pickle does not serialize functions)
def tokenizer(sentences):
    return (sentence.split() for sentence in sentences)

class VocabularyAdapter:
    
    def __init__(self, vocabulary_bin):
        self._vocab = tf.contrib.learn.preprocessing.VocabularyProcessor.restore(vocabulary_bin)
    
    @property
    def size(self):
        return len(self._vocab.vocabulary_)
    
    @property
    def vector_length(self):
        return self._vocab.max_document_length


def input_features(vector_length):
    return [
        tf.feature_column.numeric_column(
            key='context', shape=vector_length, dtype=tf.int64),
        tf.feature_column.numeric_column(
            key='context_len', shape=1, dtype=tf.int64),
        tf.feature_column.numeric_column(
            key='utterance', shape=vector_length, dtype=tf.int64),
        tf.feature_column.numeric_column(
            key='utterance_len', shape=1, dtype=tf.int64),
        tf.feature_column.numeric_column(
            key='label', shape=1, dtype=tf.int64),
    ]


def input_fn(name, filenames, features, batch_size, num_epochs=None):
    example_features = tf.feature_column.make_parse_example_spec(features)

    batch_example = tf.contrib.learn.read_batch_record_features(
        file_pattern=filenames,
        features=example_features,
        batch_size=batch_size,
        num_epochs=num_epochs,
        name='read_batch_record_features_' + name
    )

    batch_target = batch_example.pop('label')

    return batch_example, batch_target

**Training**

In [ ]:
import os

HOME_DIR = 'ubuntu'
DATA_DIR = os.path.join(HOME_DIR, 'data')
VOCAB_BIN = os.path.join(DATA_DIR, 'vocabulary.bin')
TRAIN_TFR = os.path.join(DATA_DIR, 'train.tfrecords')
VALID_TFR = os.path.join(DATA_DIR, 'valid.tfrecords')
TEST_TFR = os.path.join(DATA_DIR, 'test.tfrecords')

if not os.path.isfile(VOCAB_BIN):
    raise Exception('File not found: {}'.format(VOCAB_BIN))

if not os.path.isfile(TRAIN_TFR):
    raise Exception('File not found: {}'.format(TRAIN_TFR))

if not os.path.isfile(VALID_TFR):
    raise Exception('File not found: {}'.format(VALID_TFR))

if not os.path.isfile(TEST_TFR):
    raise Exception('File not found: {}'.format(TEST_TFR))

In [ ]:
vocab = VocabularyAdapter(VOCAB_BIN)
features = input_features(vocab.vector_length)

In [ ]:
params = {
    'vocab_size': vocab.size,
    'embed_size': 100,
    'hidden_size': 256,
    'learning_rate': 0.001,
    'optimizer': 'Adam',
    'batch_size': 128,
    'num_epochs': 5,
}

input_fn_train = lambda: input_fn('train', [TRAIN_TFR], features, params['batch_size'], 1)
input_fn_valid = lambda: input_fn('valid', [VALID_TFR], features, 16, 1)
input_fn_test = lambda: input_fn('test', [TEST_TFR], features, 16, 1)

In [ ]:
import shutil

def remove_dir(path):
    if os.path.isdir(path):
        shutil.rmtree(path)

MODEL_DIR = os.path.join(HOME_DIR, 'model')

remove_dir(MODEL_DIR)

In [ ]:
estimator = tf.estimator.Estimator(
    model_fn=model_fn,
    model_dir=MODEL_DIR,
    params=params)

estimator

In [ ]:
%%time

for epoch in range(1, params['num_epochs'] + 1):
    print('[ Epoch {} ]\n'.format(epoch))
    print('Training...\n')
    %time estimator.train(input_fn_train, steps=None)
    
    print('Validation...\n')
    %time eval_result = estimator.evaluate(input_fn_valid, steps=None)
    print(eval_result, '\n')
    print()